In [1]:
# #Set up google colab working environment
# %cd ~/../content
# %rm -rf Colombian-peso-counterfeit-detection
# !git clone https://github.com/zrosen3/Colombian-peso-counterfeit-detection.git 
# %cd /content/Colombian-peso-counterfeit-detection/Code  

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import common_functions as cf

# CNN

In [3]:
def Reshape5D(time_steps: int = 1, rows: int = 480, cols: int = 640, channels: int = 1):
    def Helper(x, y):
        # Reshape the element to have dimensions (time_steps, rows, cols, channels)
        return tf.reshape(x, [-1, time_steps, rows, cols, channels]), y
    return Helper

In [4]:
def MakeModel(class_names: list) -> tf.keras.Sequential:
    """
    Simple straight forward CNN model. this is just for simplicity and testing
    atm. I will make it more modular later once I know what we are doing

    Args:
        class_names: list of the classification names
        conv_layers: list of how many filters each convolutional layer should use
        layers: list with the sizes of each hidden layer
    
    Returns:
        `tf.keras.Sequential` - a constructed tf model
    """
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Rescaling(1./255))
    model.add(tf.keras.layers.ConvLSTM2D(16, 3, kernel_regularizer='l2', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(rate=0.4))
    model.add(tf.keras.layers.Dense(len(class_names))) # output

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model
    

## Load Data

In [5]:
train_ds, test_ds, validation_ds, class_names = cf.LoadData(path="../Data/Augmented/", batch_size=8)

Found 24958 files belonging to 13 classes.
Found 8320 files belonging to 13 classes.
Found 8320 files belonging to 13 classes.


In [6]:
model = MakeModel(class_names)
history = model.fit(
    train_ds.map(Reshape5D()), 
    validation_data=validation_ds.map(Reshape5D()),
    epochs=10
)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/conv_lstm2d/while/clip_by_value_1/Minimum' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/

# Evaluate

In [ ]:
cf.EvaluateModel(model, test_ds.map(Reshape5D()), history)

130/130 [==============================] - 4s 30ms/step - loss: 0.1522 - accuracy: 0.9596
[test loss, test accuracy]: [0.15219970047473907, 0.9596154093742371]


# Results

grayscale, no augmentation, hyperparameterized = 0.998 searched, 0.995 trained, 0.9957 test